# `XRADD`: Using `xarray` in `DiscreteDistribution`

First we import relevant libraries and tools, including the new `XRADiscreteDistribution` class.

In [1]:
from HARK.distribution import (
    MeanOneLogNormal,
    combine_indep_dstns,
    XRADiscreteDistribution,
)
import numpy as np


We create a distribution of shocks to income from continuous distributions. 

In [2]:
PermShkDstn = MeanOneLogNormal().approx(200)
TranShkDstn = MeanOneLogNormal().approx(200)
IncShkDstn = combine_indep_dstns(PermShkDstn, TranShkDstn)


Taking the components of `IncShkDstn`, we can now create a `XRADiscreteDistribution` object. As a demonstration of additional features, we can add a name attribute to the `XRADD` object, as well as named dimensions and coordinates. 

In [3]:
x_dist = XRADiscreteDistribution(
    IncShkDstn.pmf,
    IncShkDstn.X,
    name="Distribution of Shocks to Income",
    dims=("rv", "x"),
    coords={"rv": ["perm_shk", "tran_shk"]},
)


The underlying object and metadata is stored in a `xarray.DataArray` object which can be accessed using the `.xarray` attribute.

In [4]:
x_dist.xarray


<xarray.DataArray 'Distribution of Shocks to Income' (rv: 2, x: 40000)>
array([[ 0.03491194,  0.03491194,  0.03491194, ..., 11.50651472,
        11.50651472, 11.50651472],
       [ 0.03491194,  0.0530801 ,  0.06439949, ...,  5.72401042,
         6.9659349 , 11.50651472]])
Coordinates:
  * rv       (rv) <U8 'perm_shk' 'tran_shk'
Dimensions without coordinates: x
Attributes:
    pmf:      [2.5e-05 2.5e-05 2.5e-05 ... 2.5e-05 2.5e-05 2.5e-05]
    seed:     0
    RNG:      RandomState(MT19937)

### Taking the Expected Value of `XRADD` objects.

Taking the expectation of a `XRADD` object is straightforward using the own `calc_expectation` method. 

In [5]:
x_dist.calc_expectation()


array([1., 1.])

As in the `DiscreteDistribution`, we can provide a function and arguments to the `calc_expectation` method.

In [6]:
aGrid = np.linspace(0, 20, 100)
R = 1.03


In [7]:
%%timeit
x_dist.calc_expectation(lambda x, a, R: R * a / x[0] + x[1], aGrid, R)


12.6 ms ± 260 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Using functions with labels to take expresive expectations. 

The main difference is that the `calc_expectation` method of `XRADD` objects can take a function that uses the labels of the `xarray.DataArray` object. This allows for clearer and more expresive mathematical functions and transition equations. Surprisingly, using a function with labels does not add much overhead to the function evaluation. 

In [8]:
%%timeit
x_dist.calc_expectation(lambda x, a, R: R * a /x["perm_shk"] + x["tran_shk"], aGrid, R, labels=True)

12.7 ms ± 565 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
